
# 🧪 **Optuna Results Analysis - Intermediate Tests with GLM, GAM and SR**  

The objective of this notebook is to analyze the intermediate results obtained by Optuna optimization of the three trained models: Generalized Linear Model (GLM), Generalized Additive Model (GAM) and Symbolic Regression (SR).

***

### 📚 **Importing libraries**

In [1]:
import optuna
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_timeline

import pandas as pd
import numpy as np
import math
import plotly.express as px
import plotly.graph_objects as go

c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **Generalized Linear Model (GLM)**

In [2]:
study_name = "optimization_clusters_glm_teste1751900564.8571913"
storage_url = "sqlite:///../Optuna_files/optimization_glm_bayesian.db"

study_glm = optuna.load_study(study_name=study_name, storage=storage_url)


##### **Best trial**

In [3]:
print(f"The best trial had RMSE equal to {study_glm.best_value:.2f}")

The best trial had RMSE equal to 13.84


In [4]:
print(f"The best params were: {study_glm.best_params}")

The best params were: {'clusterer': 'kmeans', 'n_clusters': 10, 'distribution': 'gaussian', 'link_gaussian': 'identity'}


##### **Optimization history**

In [5]:
df_glm = study_glm.trials_dataframe()
df_filter_glm = df_glm[(df_glm["state"] != "PRUNED") & (df_glm["value"] <= 10e1)]

df_filter_glm = df_glm[(df_glm["state"] != "PRUNED") & (df_glm["value"] <= 10e1)]

df_filter_glm["n_clusters"] = df_filter_glm["params_n_clusters"].fillna(df_filter_glm["params_n_clusters_bkmeans"])


C:\Users\julia24002\AppData\Local\Temp\ipykernel_24560\2447139521.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter_glm["n_clusters"] = df_filter_glm["params_n_clusters"].fillna(df_filter_glm["params_n_clusters_bkmeans"])


This plot shows the study optimization history, by showing the objective value for each Optuna trial. Besides that, we can see the clusterer that was used and the number of clusters chosen.

In [6]:
fig = px.scatter(df_filter_glm, 
                 x="number", 
                 y="value", 
                 symbol="params_clusterer",
                 color="n_clusters",
                 title="Optimization History - GLM",)
fig.update_traces(marker=dict(size=12))
fig.update_coloraxes(
    colorbar_title_side="top",
    colorbar=dict(len=0.6)  # reduz altura da barra de cor
)

fig.show()


Now we can clearly see that most attempts are still close to 15, which makes sense considering that the best attempt reached a value of 13.83. Besides that, it is evident that the prefered cluster was kmeans and more clusters led to a better result.

##### **Hyperparameter Importances**

Plot hyperparameter importances based on the Optuna attempts.

In [7]:
plot_param_importances(study_glm)

Distribution seems like the most important feature for Optuna Optimization.

##### **Timeline plot**

Plot the timeline of a study, indicating complete and pruned trials.

In [8]:
plot_timeline(study_glm)


Many prunned trials!

### **Generalized Additive Model (GAM)**

In [9]:
study_name = "optimization_clusters_gam_teste17_07_2025_08_21_14" 

storage_url = f"sqlite:///../Optuna_files/optimization_gam_bayesian.db"

study_gam = optuna.load_study(study_name=study_name, storage=storage_url)

##### **Best trial**

In [10]:
print(f"The best trial had RMSE equal to {study_gam.best_value:.2f}")

The best trial had RMSE equal to 10.62


In [11]:
print(f"The best params were {study_gam.best_params}")

The best params were {'clusterer': 'bisecting_kmeans', 'n_clusters_bkmeans': 20, 'bisecting_strategy': 'largest_cluster', 'distribution': 'normal', 'link_gaussian': 'identity', 'lam': 0.4063514687351126, 'n_splines': 22}


##### **Optimization history**

First, it´s necessary to remove outliers and concat two params that are used for the same thing: "params_n_clusters" and "params_n_clusters_bkmeans". Both are used to count the number of clusters, but for different clusteres.

In [12]:
df_gam = study_gam.trials_dataframe()
df_filter_gam = df_gam[(df_gam["state"] != "PRUNED") & (df_gam["value"] <= 10e1)]

df_filter_gam = df_gam[(df_gam["state"] != "PRUNED") & (df_gam["value"] <= 10e1)]

df_filter_gam["n_clusters"] = df_filter_gam["params_n_clusters"].fillna(df_filter_gam["params_n_clusters_bkmeans"])

C:\Users\julia24002\AppData\Local\Temp\ipykernel_24560\3863690123.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
fig = px.scatter(df_filter_gam, 
                 x="number", 
                 y="value", 
                 symbol="params_clusterer",
                 color="n_clusters",
                 title="Optimization History - GAM",)
fig.update_traces(marker=dict(size=12))
fig.update_coloraxes(
    colorbar_title_side="top",
    colorbar=dict(len=0.6)  # reduz altura da barra de cor
)


fig.show()

The behavior is very similar to GLM. But two differences can be noticed: the values ​​are smaller and there are fewer attempts that weren´t pruned.

##### **Hyperparameter Importances**

In [14]:
plot_param_importances(study_gam)

In the case of GAM, the most important hyperparameter was the number of splines per feature.

##### **Timeline plot**

In [15]:
plot_timeline(study_gam)

The time was longer compared with GLM, but it appears there were fewer pruned trials.

### **Symbolic Regression (SR)**

In [16]:
study_name = "optimization_clusters_gam_teste15_07_2025_15_40_36" 
# The first version of script_sr.py defined the name with "gam" word. This was fixed in the new version!
from pathlib import Path

storage_url = f"sqlite:///../Optuna_files/optimization_sr.db"


study_sr = optuna.load_study(study_name=study_name, storage=storage_url)

#### **Best trial**

In [17]:
print(f"The best trial had RMSE equal to {study_sr.best_value:.2f}")

The best trial had RMSE equal to 17.49


In [18]:
print(f"The best params were: {study_sr.best_params}")

The best params were: {'clusterer': 'bisecting_kmeans', 'n_clusters_bkmeans': 17, 'bisecting_strategy': 'largest_cluster', 'n_iterations': 23, 'maxsize': 29, 'maxdepth': 14, 'n_binary_operators': 9, 'binary_operators_0': '*', 'binary_operators_1': '^', 'binary_operators_2': '*', 'binary_operators_3': '*', 'binary_operators_4': '+', 'binary_operators_5': '*', 'binary_operators_6': '+', 'binary_operators_7': '+', 'binary_operators_8': '-', 'n_unary_operators': 19, 'unary_operators_0': 'sin', 'unary_operators_1': 'sin', 'unary_operators_2': 'neg', 'unary_operators_3': 'tan', 'unary_operators_4': 'cube', 'unary_operators_5': 'cos', 'unary_operators_6': 'sqrt', 'unary_operators_7': 'inv', 'unary_operators_8': 'sin', 'unary_operators_9': 'tan', 'unary_operators_10': 'abs', 'unary_operators_11': 'exp', 'unary_operators_12': 'exp', 'unary_operators_13': 'log', 'unary_operators_14': 'log', 'unary_operators_15': 'sqrt', 'unary_operators_16': 'abs', 'unary_operators_17': 'neg', 'unary_operators_1

#### **Optimization story**

In [19]:
df_sr = study_sr.trials_dataframe()
df_filter_sr = df_sr[(df_sr["state"] != "PRUNED") & (df_sr["value"] <= 10e1)]

df_filter_sr = df_sr[(df_sr["state"] != "PRUNED") & (df_sr["value"] <= 10e1)]

df_filter_sr["n_clusters"] = df_filter_sr["params_n_clusters"].fillna(df_filter_sr["params_n_clusters_bkmeans"])

C:\Users\julia24002\AppData\Local\Temp\ipykernel_24560\3303976698.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
fig = px.scatter(df_filter_sr, 
                 x="number", 
                 y="value", 
                 symbol="params_clusterer",
                 color="n_clusters",
                 title="Optimization History - SR",)
fig.update_traces(marker=dict(size=12))
fig.update_coloraxes(
    colorbar_title_side="top",
    colorbar=dict(len=0.6)  # reduz altura da barra de cor
)

fig.show()

Fewer attempts were pruned, compared to GLM and GAM. Addiotionally, more trials were performed with bisecting_kmeans, although kmeans still dominates the trials. 

##### **Hyperparameter Importances**

In [21]:
plot_param_importances(study_sr)

Maxsize appears as the most important feature, being followed by n_iterations and n_features. It´s difficult to analyse how unary operators 1 was the second most important feature, but it´s possible to notice that unary operators were most import for the predictions that binary ones.

#### **Timeline plot**

In [22]:
plot_timeline(study_sr)

[W 2025-07-18 09:05:14,769] The start and end times for Trial 100 seem to be reversed. The start time is 2025-07-16 09:50:27.571957 and the end time is 2025-07-16 03:39:39.144938.


Also many pruned trials were observed. A mistake led Optuna to consider a trial as "Running" for a long time, but we can disgard this trial for the analysis.

### 🤔 **Conclusions**

At the end of the analysis, we can conclude that all three tests resulted in studies with a high number of pruned trials. This likely occurred due to two main factors:
*  First, the supervised models used (GLM, GAM, and Symbolic Regression) can be numerically unstable, especially GLM and GAM. These models are prone to divergence depending on the chosen distribution and link function; 
* Second, the frequent selection of KMeans and Bisecting KMeans as preferred clustering methods suggests that other clustering algorithms may have led to unstable trials as well.


Regarding feature importance, in the GLM trials, the most influential parameter was the distribution, which aligns with the observed numerical instability. For GAM and Symbolic Regression, the most important parameters were, respectively, the number of splines and maxsize, both of which directly influence model complexity. In general, as observed in the notebook "Symbolic Regression Equations", more complex models tended to deliver better performance.


### 🗃️ **References!**

[1] Quick Visualization for Hyperparameter Optimization Analysis — Optuna 4.4.0 documentation. Available: <https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html>. Acess in: 9 jul. 2025.

